In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
import itertools
import time

In [ ]:
def before_after(a, total_width):
    before = (total_width-a)//2
    after = total_width-before-a
    return (before, after)


def get_pad_width(x,y,z,total_width=400):
    return (before_after(x,total_width),
            before_after(y,total_width),
            before_after(z,total_width))


def get_padded_data(data, width=400):
    x,y,z = list(data.shape)
    return np.pad(data,get_pad_width(x,y,z),'constant',constant_values=0)


def get_block_means(data, width=400, num_blocks=4):
    block_means=[]
    ix = []
    for i in range(num_blocks):
        ix.append([width // num_blocks * i, width // num_blocks * (i+1)])
    count = 0
    for i, j, k in itertools.product(ix,ix,ix):
        block_means.append(np.mean(data[i[0]:i[-1],j[0]:j[-1],k[0]:k[-1]]))
    return block_means


def get_centered_block_means(data, width=400, num_blocks=10):
    ix = []
    for i in range(num_blocks):
        half_width = width//2
        half_region = width//num_blocks//2*(i+1)
        ix.append([half_width-half_region, half_width+half_region])
    centered_means = []
    for i in ix:
        centered_means.append(np.mean(padded[i[0]:i[-1],i[0]:i[-1],i[0]:i[-1]]))
    return centered_means

In [ ]:
df = pd.read_csv('../csvs/GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

In [ ]:
blocks = 4
block_labels = ['block_' + str(x) + '_mean' for x in range(1,blocks**3+1)]
centered_blocks = 10
centered_block_labels = ['centered_block_' + str(x) + '_mean' for x in range(1,centered_blocks+1)]
new_features = block_labels + centered_block_labels
try:
  df = df.reindex(columns = df.columns.tolist() + new_features)
except: 
    pass

In [ ]:
start = time.time()
start100 = start
for i, img in enumerate(df.img):
    if i % 25 == 0 and i != 0:
        now = time.time()
        print('i = {:<10d}time for last 25 = {:<10.3g}total time = {:<10.3g}'.format(i,now-start100,now-start))
        start100 = time.time()
    padded = get_padded_data(img.get_fdata())
    region_means = get_block_means(padded)
    centered_means = get_centered_block_means(padded)
    features = region_means + centered_means
    df.loc[df.index==i,new_features] = features

print('Done with {:d} rows. Total time = {:<10.3g}'.format(len(df),time.time()-start))

In [ ]:
df.drop('img',axis=1).to_csv('../new_output/regional_features.csv')